# ABS Monthly Lending 5601

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from plotting import (
    line_plot,
    seas_trend_plot,
)

# pandas display settings
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5601.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

## Plotting

### Set-up

### Headline Indicators

In [3]:
def fix_title(title, lfooter):
    """Title beautification."""

    check = [
        "Total housing excluding refinancing",
        "Total purpose excluding refinancing",
        "New loan commitments",
        "Fixed term loans",
        "Value",
    ]
    for c in check:
        text = f"{c} ;"
        if text in title:
            title = title.replace(text, "")
            lfooter = lfooter + f"{c}. "
    title = (
        title.replace("Businesses", "")
        .replace("Business Finance", "Business Finance -")
        .replace("Households", "Households -")
        .replace(";", "")
        .replace("    ", " ")
        .replace("   ", " ")
        .replace("  ", " ")
        .strip()
    )
    return title, lfooter

In [4]:
def headline_charts() -> None:
    """Produce headline charts."""

    rows = [
        "Households ;  Housing Finance ;  Total housing excluding refinancing ;  New loan commitments ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Purchase of property ;  New loan commitments ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Construction ;  New loan commitments ;  Value ;",
        "Households ;  Personal Finance ;  Fixed term loans ;  Total purpose excluding refinancing ;  New loan commitments ;",
    ]

    for headline in rows:
        # get data
        f = pd.DataFrame()
        for series_type in "Seasonally Adjusted", "Trend":
            found = meta[
                meta[mc.stype].str.contains(series_type)
                & meta[mc.did].str.contains(headline)
            ]
            assert len(found) == 1
            row = found.iloc[0]
            series_id, units, table, did = (
                row[mc.id],
                row[mc.unit],
                row[mc.table],
                row[mc.did],
            )
            f[series_type] = (abs_dict[table])[series_id]
        f_, units = ra.recalibrate(f, units)

        # plot
        title, lfooter = fix_title(did, "Australia. ")
        seas_trend_plot(
            pd.DataFrame(f_),  # mypy
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=f"{units} / Month",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


headline_charts()

### Business

In [5]:
def business_charts() -> None:
    """Produce business charts."""

    rows = meta[
        meta[mc.did].str.contains("Business")
        & meta[mc.did].str.contains("loans")
        & meta[mc.did].str.contains("Total purpose")
    ]
    for _, row in rows.iterrows():
        series_id, units, table, did, series_type = (
            row[mc.id],
            row[mc.unit],
            row[mc.table],
            row[mc.did],
            row[mc.stype],
        )
        series, units = ra.recalibrate(abs_dict[table][series_id], units)
        title, lfooter = fix_title(did, f"Australia. {series_type} series. ")
        title = title.replace("Businesses", "").strip()
        line_plot(
            series,
            title=title,
            ylabel=f"{units} / Month",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


business_charts()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Jul 16 2024 09:31:23

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

matplotlib: 3.9.1
readabs   : 0.0.10a4
pandas    : 2.2.2

Watermark: 2.4.3



In [7]:
print("Done")

Done
